# TLDR

The goal of this notebook is to answer the following questions:
- What do the subspaces for no-retraining PCA and retraining look like?
- Does it help to retrain the subspace on high-utility outcome points? 
- Would retraining on high-util outcome points help only when the initial points do not have good coverage over the space?


The steps to answer these questions:
- load the saved `pref_data_dict` from finished experiments. Those contain the initial + EUBO-selected outcome vectors as well as their true utility value
- train the following subspaces, keeping them at the same latent dim
    - subspace 1: learned from initial batch (32 points) of Y
    - subspace 2: take 32 points with highest util values
    - subspace 3: take 32 points with lowest util values
- Then, fit outcome and util models respectively, and compare
    - outcome reconstruction error
    - util model accuracy
    - overall model accuracy
    - (later) pass as a linear projection into BOPE and see BOPE performance, but could just be similar

In [1]:
%load_ext autoreload
%autoreload 2

import itertools
import pickle
import re
import warnings
from collections import defaultdict
from dataclasses import asdict, dataclass
from typing import Dict, List, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sns
import torch 

warnings.filterwarnings("ignore")

import sys
sys.path.append('..')
sys.path.append('/home/yz685/low_rank_BOPE')
sys.path.append('/home/yz685/low_rank_BOPE/low_rank_BOPE')

from helpers.plotting_helpers import plot_performance_over_comps_multiple, plot_performance_over_comps_single, \
    plot_subspace_diagnostics_single, \
    plot_result_metric_multiple, colors_dict, marker_dict, labels_dict
from low_rank_BOPE.src.transforms import fit_pca

/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
trial_range = range(31, 51)
problems = ["8by8_rectangle_gradient_aware_area"]
parent_dir_name = "shapes_rt"


# problems = {
#     "vehiclesafety_5d3d_piecewiselinear_24_0.01": (5,24),
#     "carcabdesign_7d9d_piecewiselinear_72_0.01": (7,72),
#     "carcabdesign_7d9d_linear_72_0.01": (7,72)}
# trial_range = range(1,21)
# parent_dir_name = "cars_rt"

In [3]:
outputs = defaultdict(lambda: defaultdict(dict))
valid_trials = []

for problem in problems:

    results_folder = f'/home/yz685/low_rank_BOPE/experiments/{parent_dir_name}/{problem}/'

    for trial in trial_range:

        try:

            outputs[problem]['exp_candidate_results'][trial] = \
                list(vv for v in torch.load(results_folder + f'final_candidate_results_trial={trial}.th').values() for vv in v.values())
            
            outputs[problem]['within_session_results'][trial] = \
                list(itertools.chain.from_iterable(vv for v in torch.load(results_folder + f'PE_session_results_trial={trial}.th').values() for vv in v.values()))

            outputs[problem]['subspace_diagnostics'][trial] = \
                torch.load(results_folder + f'subspace_diagnostics_trial={trial}.th')
            
            outputs[problem]['pref_data'][trial] = \
                torch.load(results_folder + f'pref_data_trial={trial}.th')
            
            valid_trials.append(trial)
        
        except:
            print(f"Trial {trial} not finished yet, skipping for now")
            continue
    

In [5]:
example_output = outputs["8by8_rectangle_gradient_aware_area"]["pref_data"][31][('pca','EUBO-zeta')]

In [6]:
Y = example_output["Y"]
util_vals = example_output["util_vals"]

In [7]:
Y.shape

torch.Size([112, 64])

In [8]:
proj_1 = fit_pca(Y[:32], standardize=False)

In [9]:
largest_ind = torch.topk(util_vals.squeeze(1), k=32).indices
largest_Y = Y[largest_ind]

smallest_ind = torch.topk(util_vals.squeeze(1), largest=False, k=32).indices
smallest_Y = Y[smallest_ind]

In [10]:
proj_2 = fit_pca(largest_Y, standardize=False)
proj_3 = fit_pca(smallest_Y, standardize=False)

In [ ]:
# TODO: keep them at the same dimension, then compute Grassmann distance
# learn about principal angles, etc. 

# https://web.ma.utexas.edu/users/vandyke/notes/deep_learning_presentation/presentation.pdf

In [35]:
print(proj_1.shape, proj_2.shape, proj_3.shape)

torch.Size([12, 64]) torch.Size([12, 64]) torch.Size([10, 64])


In [24]:
torch.dot(proj_2[0], proj_3[0])

tensor(-0.5071, dtype=torch.float64)

In [34]:
def get_principal_angles(subspace1, subspace2):
    r"""
    
    Args:
        subspace1: `latent_dim x outcome_dim` 
        subspace2: `latent_dim x outcome_dim`

    """

    subspace1_ = torch.transpose(subspace1, -2, -1)
    subspace2_ = torch.transpose(subspace2, -2, -1)

    q1, _ = torch.linalg.qr(subspace1_)
    q2, _ = torch.linalg.qr(subspace2_)

    _, S, _ = torch.linalg.svd(torch.transpose(q1, -2, -1) @ q2)

    print('singular values: ', S)

    theta = torch.arccos(S)

    return theta

In [36]:
get_principal_angles(
    subspace1 = proj_1,
    subspace2 = proj_2
)

singular values:  tensor([0.9945, 0.9829, 0.8504, 0.7357, 0.6343, 0.5203, 0.4482, 0.3550, 0.3052,
        0.2458, 0.1163, 0.0024], dtype=torch.float64)


tensor([0.1046, 0.1852, 0.5541, 0.7441, 0.8837, 1.0236, 1.1061, 1.2079, 1.2606,
        1.3224, 1.4543, 1.5684], dtype=torch.float64)

In [37]:
get_principal_angles(
    subspace1 = proj_1[:10],
    subspace2 = proj_3
)

singular values:  tensor([0.9915, 0.9837, 0.9756, 0.9548, 0.9265, 0.8844, 0.7896, 0.6632, 0.4345,
        0.0809], dtype=torch.float64)


tensor([0.1302, 0.1810, 0.2213, 0.3019, 0.3859, 0.4856, 0.6607, 0.8457, 1.1213,
        1.4899], dtype=torch.float64)